# Avaliando nível dos reservatórios em anos de seca com NDWI com [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED?hl=pt-br)
- Foi usado como exemplo da `cheia` [12 de junho de 2019](https://epocanegocios.globo.com/Brasil/noticia/2021/06/imagens-da-nasa-mostram-antes-e-depois-da-maior-seca-no-brasil-em-quase-um-seculo.html) no Lago das Brisas (MG) e comparado com período `seca em 17 de junho de 2021`.
- O lago das Brisas é um lago localizado a cerca de 202 km de Goiânia, no estado brasileiro de Goiás. Situado no município de Buriti Alegre, o lago das Brisas tem 778 km² de área, cerca de duas vezes e meia maior que a baía de Guanabara.



# Inicializando o GEE

In [1]:
# importando o GEE e Geemap
import ee
import geemap

# inicializando GEE
geemap.ee_initialize(project='ee-enrique')

# Definindo região

In [2]:
# região de interesse
lago_das_brisas_total = ee.Geometry.Rectangle([-49.17, -18.48, -48.30, -18.19])
lago_das_brisas_zoom = ee.Geometry.Rectangle([-49.12, -18.50, -48.98, -18.32])

# criando um mapa interativo
Map = geemap.Map()

# centraliza o mapa
Map.centerObject(lago_das_brisas_total, 11)

# adicionando basemap
Map.add_basemap('SATELLITE')

# contorno da região
Map.addLayer(lago_das_brisas_total, {'color': 'yellow', 'fillColor': '00000000', 'opacity': 0.3}, 'Região')

# exibe na tela
Map

Map(center=[-18.33545255037958, -48.735000000000056], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# extraindo as coordenadas de latitude/longitude que limita a região do mapa selecionada
#regiao = Map.draw_last_feature.geometry()

# mostra as latitudes/longitudes limitantes da região
#regiao

# Mapa Lago das Brisas sem Zoom com combinação de RGB e Falsa cor para destacar água

In [13]:
# região de interesse
lago_das_brisas_total = ee.Geometry.Rectangle([-49.17-0.5, -18.48-0.5, -48.30+0.5, -18.19+0.5])
lago_das_brisas_zoom = ee.Geometry.Rectangle([-49.12, -18.50, -48.98, -18.32])

# JUNHO de 2019 - CHEIA
imagens_s2_2019_cheia = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                          .filterBounds(lago_das_brisas_total) \
                          .filterDate('2019-06-01','2019-07-01') \
                          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                          .map(lambda img: img.multiply(0.0001).copyProperties(img, img.propertyNames()))

# JUNHO de 2021 - SECA
imagens_s2_2021_seca = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                         .filterBounds(lago_das_brisas_total) \
                         .filterDate('2021-06-01','2021-07-01') \
                         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                         .map(lambda img: img.multiply(0.0001).copyProperties(img, img.propertyNames()))


# parâmetros de visualização
param_vis_rgb = {'bands': ['B4', 'B3', 'B2'], 'min': 0.03, 'max': 0.19}
param_vis_agua = {'bands': ['B12', 'B8', 'B4'], 'min': 0.03, 'max': 0.19}

# monta painel com 4 mapas
geemap.linked_maps(rows = 2,
                   cols = 2,
                   height = "200px",
                   center = [-18.35, -49.05],
                   zoom = 10.1,

                   ee_objects = [imagens_s2_2019_cheia.mean().clip(lago_das_brisas_total),
                                 imagens_s2_2021_seca.mean().clip(lago_das_brisas_total),

                                 imagens_s2_2019_cheia.mean().clip(lago_das_brisas_total),
                                 imagens_s2_2021_seca.mean().clip(lago_das_brisas_total)],

                   vis_params = [param_vis_rgb,
                                 param_vis_rgb,
                                 param_vis_agua,
                                 param_vis_agua],

                   labels = ['CHEIA: RGB',
                             'SECA: RGB',
                             'CHEIA: FALSA COR',
                             'SECA: FALSA COR'],

                   label_position = "topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [9]:
# mostra datas das imagens
imagens_s2_2019_cheia

Output hidden; open in https://colab.research.google.com to view.

In [10]:
# mostra datas das imagens
imagens_s2_2021_seca

Output hidden; open in https://colab.research.google.com to view.

# Mapa Lago das Brisas com Zoom e calculo da àrea antes com CHEIA (junho 2019) e depois SECA (junho 2021)

In [7]:
#===============================================================================================#
#                                     REGIÃO DE INTERESSE
#===============================================================================================#
lago_das_brisas_total = ee.Geometry.Rectangle([-49.17, -18.48, -48.30, -18.19])
lago_das_brisas_zoom = ee.Geometry.Rectangle([-49.12, -18.50, -48.98, -18.32])

#===============================================================================================#
#                                     CARREGA OS DADOS
#===============================================================================================#
# JUNHO de 2019 - CHEIA
imagens_s2_2019_cheia = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                          .filterBounds(lago_das_brisas_total) \
                          .filterDate('2019-06-01','2019-07-01') \
                          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                          .map(lambda img: img.multiply(0.0001).copyProperties(img, img.propertyNames()))

# JUNHO de 2021 - SECA
imagens_s2_2021_seca = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                         .filterBounds(lago_das_brisas_total) \
                         .filterDate('2021-06-01','2021-07-01') \
                         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                         .map(lambda img: img.multiply(0.0001).copyProperties(img, img.propertyNames()))

#===============================================================================================#
#                                     CALCULA NDWI
#===============================================================================================#
# calcula NDWI
def ndwi(img):
    return img.addBands(img.normalizedDifference(['B3', 'B8']).rename('ndwi')) \
              .copyProperties(img, ['system:time_start']) \
              .set('date', img.date().format('YYYY-MM-dd'))

# aplica a função de NDWI em cada imagem
imagens_s2_2019_cheia_ndwi = imagens_s2_2019_cheia.map(ndwi)
imagens_s2_2021_seca_ndwi = imagens_s2_2021_seca.map(ndwi)

# cria as máscaras para identificar valores máximos de NDWI
mask_agua_cheia = imagens_s2_2019_cheia_ndwi.select('ndwi').max().gt(0).selfMask().rename('cheia').set('date', '2019-06-01')
mask_agua_seca = imagens_s2_2021_seca_ndwi.select('ndwi').max().gt(0).selfMask().rename('seca').set('date', '2021-06-01')

#===============================================================================================#
#                               CALCULA ÁREA COM ÁGUA
#===============================================================================================#
# junta as imagens da CHEIA e SECA
mask_agua_cheia_seca = mask_agua_cheia.clip(lago_das_brisas_total).addBands(mask_agua_seca.clip(lago_das_brisas_total))

# cálcula a área de cada pixel em km²:
# 1. Multiplica pela área do pixel em m²
# 2. Divide por 1.000.000 para converter para km²
imagem_area_km2 = mask_agua_cheia_seca.multiply(ee.Image.pixelArea()).divide(1000000)

# acumula todos os pixels que estão em km²
soma_area_km2 = imagem_area_km2.reduceRegion(reducer=ee.Reducer.sum(),
                                             geometry=lago_das_brisas_total,
                                             scale=10,
                                             maxPixels=1e13,
                                             bestEffort=True)

# obtém os valores em km²
area_cheia_km2 = soma_area_km2.get('cheia').getInfo()
area_seca_km2 = soma_area_km2.get('seca').getInfo()

print(f'Área período de CHEIA: {area_cheia_km2:.1f} km²')
print(f'Área período de SECA: {area_seca_km2:.1f} km²')
dif = area_cheia_km2 - area_seca_km2
print(f'DIMINUIÇÃO da àrea: {dif:.1f} km²')

#===============================================================================================#
#                           PLOTA MAPA DE RGB E MASCARA DE ÁGUA
#===============================================================================================#
# parâmetro de visualização
param_vis_mask = {'palette' : ['blue'], 'min': 0, 'max': 1}

# monta painel com 4 mapas
geemap.linked_maps(rows = 1,
                   cols = 2,
                   height = "200px",
                   center = [-18.40, -49.05],
                   zoom = 9.45,
                   ee_objects = [mask_agua_cheia.clip(lago_das_brisas_total),
                                 mask_agua_seca.clip(lago_das_brisas_total)],
                   vis_params = [param_vis_mask,
                                 param_vis_mask],
                   labels = ['JUNHO 2019: MASK',
                             'JUNHO 2021: MASK'],
                   label_position = "topright")

Área período de CHEIA: 453.5 km²
Área período de SECA: 321.7 km²
DIMINUIÇÃO da àrea: 131.9 km²


GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…